In [2]:
import tensorflow as tf
from google.protobuf.json_format import MessageToJson
import json

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
path_to_tfrecord = '/datadrive/emammal_tfrecords/eMammal_loc_splits_20180929/train-00104-of-00110'

In [16]:
i = 0
example = ''

# note that the 'image/class/label' is at the image level - look at the bbox level instead
for example in tf.python_io.tf_record_iterator(path_to_tfrecord):
    
    result = tf.train.Example.FromString(example)
    jsonMessage = MessageToJson(tf.train.Example.FromString(example))

    jsonExample = json.loads(jsonMessage)
    if 'value' not in jsonExample['features']['feature']['image/object/class/label']['int64List']:
        print(jsonExample['features']['feature']['image/object/class/label']['int64List'])
    
    elif jsonExample['features']['feature']['image/object/class/label']['int64List']['value'][0] == '2':
        print('Found example')
        print(jsonExample['features']['feature']['image/object/class/label']['int64List']['value'])
    
    i += 1

{}
{}
Found example
['2']
{}
{}
{}


In [5]:
color_space = jsonExample['features']['feature']['image/colorspace']['bytesList']['value'][0]

In [6]:
color_space

'UkdC'

## Upload to blob storage

In [17]:
from azure.storage.blob import BlobServiceClient
import json
import os
from tqdm import tqdm

In [18]:
os.environ["AZ_STORAGE_KEY"] = ''

In [19]:
key = os.environ["AZ_STORAGE_KEY"]
blob_service = BlobServiceClient(account_url='emammaltrain.blob.core.windows.net', credential=key)

In [20]:
tfrecords_dir = '/datadrive/emammal_tfrecords/eMammal_loc_splits_20180929'
container_name = 'tf20180929'

container_client = blob_service.get_container_client(container_name)
for tfrecord in tqdm(os.listdir(tfrecords_dir)):
    if not tfrecord.startswith('failed'):
        local_path = os.path.join(tfrecords_dir, tfrecord)
        with open(local_path, 'rb') as f:
            container_client.upload_blob(name=tfrecord, data=f)

100%|██████████| 141/141 [25:44<00:00, 10.95s/it]
